In [ ]:
!pip install git+https://github.com/huggingface/transformers datasets seqeval

In [ ]:
import pandas as pd
import os



from sklearn.model_selection import train_test_split

df = pd.read_json("es-train.json", lines=True)
#df_test = pd.read_json("test.json", lines=True)

del df["id"]
X = df.text
y = df.label





In [ ]:
import os
%env MY_SEED=42
seed = int(os.environ['MY_SEED'])


In [ ]:
#from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
#from sklearn.feature_extraction.text import TfidfVectorizer

# X_resampled, y_resampled = ADASYN(random_state=42).fit_resample(X_tfid, y)

train, test = train_test_split(df , test_size=0.2, random_state=seed)
#ros = RandomOverSampler(random_state=seed)

#X_train, y_train = ros.fit_resample(train.text.to_numpy().reshape(-1, 1), train.label)

#train = pd.DataFrame(columns=["text", "label"])
#train.text = X_train.flatten()
#train.label = y_train

"""
train = pd.DataFrame(columns=["text", "label"])
train.text = df.text
train.label = df.label
test = pd.DataFrame(columns=["text", "label"])
test.text = df_test.text"""

In [ ]:
# import nltk
# nltk.download('punkt')


# def sent_token(txt):
#   l = nltk.sent_tokenize(txt)
#   return [" ".join(l[:len(l)//2]), " ".join(l[len(l)//2:])]

In [ ]:
# train.to_json("train-en-updated.json", orient="records", lines=True)
# test.to_json("test-en-updated.json", orient="records", lines=True)

In [ ]:
!rm run_glue.py*
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/pytorch/text-classification/run_glue.py

In [ ]:
# cache_iterator_len = filter(lambda x: x[1] > 600, cache)
# cache_iterator_val = filter(lambda x: x[0].label == 1, cache)
# lcache_fil_len = list(cache_iterator_len)
# lcache_fil_v = list(cache_iterator_val)


In [ ]:
# from tqdm import tqdm
# import textwrap
# from transformers import BertTokenizer

# df = pd.read_json("en-train.json", lines=True)

# del df["id"]
# X = df.text
# y = df.label

# tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
# dfc = df.copy()

# cache = []
# for i, v in tqdm(dfc.iterrows(), total=dfc.shape[0]):
#   input = tokenizer.encode(v.text)
#   tok = tokenizer.tokenize(tokenizer.decode(input))
#   if len(tok) > 512:
#     cache.append((v, len(tok)))
    

In [ ]:
#def modify(txt):
#  return textwrap.wrap(txt, len(txt)/3)[1]
#
#for d, i in cache: 
#  new_text = modify(v.text)
#
#  if False:
#    input = tokenizer.encode(new_text)
#    tok = tokenizer.tokenize(tokenizer.decode(input))
#    if len(tok) > 256:
#      pass
# 
#  dfc.loc[d._name, "text"] = new_text


In [ ]:
#def modify(txt):
#  return textwrap.wrap(txt, len(txt)/3)[1]


# dfc = df.copy()

# for d, i in cache: 
#   new_text_list = sent_token(d.text)


#   if False:
#     input = tokenizer.encode(new_text)
#     tok = tokenizer.tokenize(tokenizer.decode(input))
#     if len(tok) > 256:
#       pass
 
#   dfc.drop(d._name, inplace=True)
#   for txt in new_text_list:
#     dfc = dfc.append({"text": txt, "label": d.label}, ignore_index=True)


In [ ]:
!rm train-en-updated.json test-en-updated.json
train.to_json("train-en-updated.json", orient="records", lines=True)
test.to_json("test-en-updated.json", orient="records", lines=True)

In [ ]:
!rm -rf ./test_file/ ./runs/

In [ ]:
%env MY_MODEL=bert-base-cased
%env MY_EPOCH=5
%env MY_BATCH=32
%env MY_METHOD=RandomOverSampling-auto

In [ ]:
!env | grep MY

In [ ]:
!python run_glue.py \
  --model_name_or_path $MY_MODEL \
  --train_file ./train-en-updated.json \
  --validation_file ./train-en-updated.json \
  --output_dir ./test_file \
  --num_train_epochs $MY_EPOCH \
  --do_train \
  --per_device_train_batch_size $MY_BATCH

In [ ]:
!python run_glue.py \
  --model_name_or_path ./test_file/ \
  --train_file ./train-en-updated.json \
  --validation_file ./test-en-updated.json \
  --test_file ./test-en-updated.json \
  --output_dir ./test_file \
  --per_device_train_batch_size $MY_BATCH \
  --do_predict

In [ ]:
#from transformers import BertForSequenceClassification


#model = BertForSequenceClassification.from_pretrained("./test_file/")
#model.,

from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification, AutoModelForSequenceClassification
from transformers import PretrainedConfig

from transformers import TokenClassificationPipeline, TextClassificationPipeline

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

model = AutoModelForSequenceClassification.from_pretrained("./test_file/")

config = PretrainedConfig.from_pretrained("./test_file/")

pipeline = pipeline('sentiment-analysis', device=0, model=model, tokenizer=tokenizer, config=config)

nlp_grouped = TextClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
)



In [ ]:
"""from collections import Counter
preds = []


k = 0
for i in list(test.text):
  k += 1
  if k % 24 == 0:
    print(k)
  res = pipeline(i, truncation=True, max_length=512)
  preds.append(res[0])


preds
"""



In [ ]:
"""import json
df_test = pd.read_json("test.json", lines=True)

with open("submission.json", "w") as f:
  for i, row in df_test.iterrows():
    if preds[i]["label"] == "LABEL_1":
      pred = 1
    else:
      pred = 0
    j = {"prediction": pred, "id":row["id"]}
    f.write(json.dumps(j) + "\n")
"""

In [ ]:
import pandas as pd

pred = pd.read_csv("./test_file/predict_results_None.txt", sep="\t")
pred

In [ ]:
from sklearn.metrics import f1_score, recall_score, precision_score

f1macro_out = f1_score(list(test.label), list(pred.prediction), average="macro")
recall_out = recall_score(list(test.label), list(pred.prediction), average=None)
recall_out_avg = recall_score(list(test.label), list(pred.prediction))


recall_out_zeros = recall_out[0]
recall_out_ones = recall_out[1]

precision_out = precision_score(list(test.label), list(pred.prediction), average=None)
precision_out_avg = precision_score(list(test.label), list(pred.prediction))

precision_out_zeros = precision_out[0]
precision_out_ones = precision_out[1]

print("f1-macro", f1macro_out)
print("recall_avg", recall_out_avg)
print("recall_zeros", recall_out_zeros, "recall_ones", recall_out_ones)
print("precision_avg", precision_out_avg)
print("precision_zeros", precision_out_zeros, "precision_ones", precision_out_ones)


In [ ]:
OUTNAME = "results.json "

# out = pd.DataFrame(columns=["model", "epoch", "batch", "f1-macro", "recall", "precision", "method", "seed", "date"])
import time
import json
# out = out.append({
#     "model"  : os.environ['MY_MODEL'],
#     "epoch"  : os.environ['MY_EPOCH'],
#     "batch"  : int(os.environ['MY_BATCH']),
#     "f1-macro": f1macro_out,
#     "recall": recall_out,
#     "precision": precision_out,
#     "seed"  : int(os.environ['MY_SEED']),
#     "method" : os.environ['MY_METHOD'],
#     "date": time.strftime("%Y-%m-%d %H:%M"),
#     }, ignore_index=True)

out =  {
    "model"  : os.environ['MY_MODEL'],
    "epoch"  : os.environ['MY_EPOCH'],
    "batch"  : int(os.environ['MY_BATCH']),
    "f1-macro": f1macro_out,
    "recall": list(recall_out),
    "precision": list(precision_out),
    "seed"  : int(os.environ['MY_SEED']),
    "method" : os.environ['MY_METHOD'],
    "date": time.strftime("%Y-%m-%d %H:%M"),
    "prediction": list(pred.prediction),
    }


with open(OUTNAME, "a+") as f:
  f.write(json.dumps(out))
  f.write("\n")


